<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<div dir='rtl'>
<h1>
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند.

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید.
</p>
</div>

In [1]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Union

In [2]:
import nltk
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید.
</p>
</div>

In [3]:
# load the dataset
dataset = pd.read_csv("data.csv")
dataset.columns[1:]

Index(['title', 'abstract', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [4]:
# import nltk
import spacy
from typing import Optional

class Preprocessor:
    instance:Optional['Preprocessor'] = None
    @staticmethod
    def get_instance(stopwords_path):
      if Preprocessor.instance == None:
        Preprocessor.instance = Preprocessor(stopwords_path = None)
      return Preprocessor.instance
    def __init__(self, stopwords_path):
        # Create a variable of stop words.
        self.stopwords = nltk.corpus.stopwords.words('english')
#         with open(stopwords_path, 'r') as f:
#             self.stopwords.extend(f.read().splitlines())

        # Load the spacy model.
        self.nlp = spacy.load('en_core_web_sm')

    def preprocess(self, text):
        # The main function of the class.
        text = self.remove_links(text)
        text = self.normalize(text)
        words = self.word_tokenize(text)
        words = self.remove_stopwords(words)
        return words

#     def normalize(self, text):
#         # Normalize text (lower case, stemming, lemmatization, etc.)
#         text = text.lower()
#         doc = self.nlp(text)
#         words = [token.lemma_ for token in doc]
#         return ' '.join(words)

    def normalize(self, text):
        # Normalize text (lower case, stemming, lemmatization, etc.)
        text = text.lower()
        text = self.remove_punctuations(text)
        text = self.lemmatize(text)
        text = self.stemm(text)
        return text

    def lemmatize(self, text):
        # Lemmatize text
        doc = self.nlp(text)
        text = ' '.join([token.lemma_ for token in doc])
        return text

    def stemm(self, text):
        # Stem text
        stemmer = nltk.stem.PorterStemmer()
        words = nltk.word_tokenize(text)
        text = ' '.join([stemmer.stem(word) for word in words])
        return text

    def remove_links(self, text):
        # Remove links
        import re
        text = re.sub(r'http\S+', '', text)
        return text

    def remove_punctuations(self, text):
        # Remove punctuations
        import string
        text = text.translate(str.maketrans('', '', string.punctuation))
        return text

    def word_tokenize(self, text):
        # Tokenize text
        words = nltk.word_tokenize(text)
        return words

    def remove_stopwords(self, words):
        # Remove stopwords
        words = [word for word in words if word not in self.stopwords]
        return words

In [5]:
# preprocess
def preprocess_dataset(dataset, preprocessor):
    null_data = dataset.isnull()
    for col in dataset.columns[1:]:
        for i in range(len(dataset)):
            if null_data.loc[i, col]:
              continue
            else:
              dataset.at[i, col] = preprocessor.preprocess(dataset.loc[i, col])

p = Preprocessor.get_instance(None)
preprocess_dataset(dataset, p)

In [6]:
# Identify null values
null_data = dataset.iloc[:, 1:].isnull()

# Create a dictionary representation of the corpus
doc_ids = dataset.iloc[:, 0].tolist()
corpus = {doc_id: {col: dataset.at[i, col] for col in dataset.columns[1:] if not null_data.at[i, col]}
          for i, doc_id in enumerate(doc_ids)}

<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید.
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید.
</p>
</div>

In [7]:
from collections import defaultdict

def construct_positional_indexes(corpus: dict) -> dict:
    """
    Constructs positional indexes for the processed data by inserting words into a trie
    and creating positional indexes and posting lists.

    Parameters
    ----------
    corpus : dict
        The processed data.

    Returns
    -------
    dict
        The constructed positional indexes.
    """

    target_values = ['title', 'abstract']
    positional_index = defaultdict(lambda: {target: defaultdict(lambda: []) for target in target_values})

    for doc_id, doc_content in corpus.items():
        for target in target_values:
            tokens = doc_content.get(target, [])
            for i, token in enumerate(tokens):
                positional_index[token][target][doc_id].append(i)

    return positional_index

docs = construct_positional_indexes(corpus)

<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم.

در مدل‌های احتمالاتی اساس کار محاسبه
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش
Okapi25
طراحی می‌کنید.
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [8]:
def get_tf(token: str, positional_index: dict, doc_id: str, target=None) -> int:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.
    target : str, optional
        The target (e.g., 'title' or 'abstract') within the document. If None, considers both 'title' and 'abstract'.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """

    if target:
        return len(positional_index.get(token, {}).get(target, {}).get(doc_id, []))

    return len(positional_index.get(token, {}).get('abstract', {}).get(doc_id, []) +
               positional_index.get(token, {}).get('title', {}).get(doc_id, []))

get_tf('deep', docs, '40ea606185b59cd07b456cb1022d64bf41f5538d')

0

In [9]:
# import numpy as np

def get_idf(token: str, docs= docs) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.
    docs : dict
        The document collection containing positional indexes.

    Returns
    -------
    float
        The IDF of the given token.
    """

    total_documents = len(doc_ids)
    doc_occurrences = sum(1 for doc_index in docs[token]['abstract'].keys() | docs[token]['title'].keys())

    return np.log(total_documents / (doc_occurrences + 1))

get_idf('deep', docs)

2.4915576597870803

<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [10]:


def generate_tfidf_list(corpus: dict, positional_index: dict) -> list[dict[str, float]]:
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.

    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.

    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """

    # TODO: Write the function to generate the list of dictionaries with TF-IDF scores.
    all_tf_idf = []

    all_tokens = set()
    documens_tokens = {}
    for doc_id in doc_ids:
        documens_tokens[doc_id] = set()
        for nn, t in corpus[doc_id].items():
            documens_tokens[doc_id] = documens_tokens[doc_id].union(set(t))

        all_tokens = all_tokens.union(documens_tokens[doc_id])

    idf = {}
    for token in all_tokens:
        idf[token] = get_idf(token, docs)

    for doc_id in doc_ids:
        tf_idf = {}

        for token in documens_tokens[doc_id]:
            tf_idf[token] = idf[token] * get_tf(token, positional_index, doc_id)

        all_tf_idf.append(tf_idf)

    return all_tf_idf

tfidf = generate_tfidf_list(corpus, docs)



In [11]:


import math

def show_document_vector(vector: list[dict[str, float]], doc_index: int) -> None:
    """
    Prints the non-zero weights and corresponding terms for a document represented as a vector.

    Parameters
    ----------
    vector : list of dict of str:float
        The list of document vectors, where each document vector is a dictionary with term weights.
    doc_index : int
        The document index for which to display the vector.

    Returns
    -------
    None
    """
    document_vector = vector[doc_index]
    sorted_terms = sorted(document_vector.items(), key=lambda x: x[1], reverse=True)
    length = math.sqrt(sum(weight ** 2 for _, weight in sorted_terms))
    normalized = {term: tfidf / length for term, tfidf in sorted_terms}

    for term, tfidf in sorted_terms:
        print(f'{term.ljust(16)}\t{tfidf}\t(normalized: {normalized[term]})')

show_document_vector(tfidf, 0)

carbid          	8.036735104266644	(normalized: 0.38308004250088257)
sic             	7.343587923706697	(normalized: 0.3500404004144708)
grind           	6.9381228155985335	(normalized: 0.33071344875667147)
ultrason        	6.783972135771275	(normalized: 0.3233656827529847)
vibrat          	6.021832083724378	(normalized: 0.28703741763762086)
ceram           	6.021832083724378	(normalized: 0.28703741763762086)
silicon         	5.957293562586807	(normalized: 0.2839611162416474)
amplitud        	5.511006459958388	(normalized: 0.26268833817636966)
minor           	5.2035217602104265	(normalized: 0.24803173318446375)
ground          	5.146363346370478	(normalized: 0.24530721292604862)
face            	4.3604344323595665	(normalized: 0.20784502487631001)
surfac          	3.6547084695927614	(normalized: 0.1742058009497733)
analysi         	1.7704837726926128	(normalized: 0.0843921057607259)


<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش
term-at-a-time
و هم به روش
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

In [12]:
def calculate_Lave(targets, corpus, doc_ids):
  Lave = {}

  for target in targets:
    section_lengths = []

    for doc_id in doc_ids:
        if target in corpus[doc_id]:
            section_lengths.extend(len(section) for section in corpus[doc_id][target])

    Lave[target] = sum(section_lengths) / len(section_lengths) if section_lengths else 0
    return Lave

# Calculate Lave
target_values = ['title', 'abstract']
Lave = calculate_Lave(target_values, corpus, doc_ids)

# targets = ['title', 'abstract']
# # Preprocess
# Lave = {}

# for target in targets:
#     lengths = []

#     for doc in doc_ids:
#         for _, section in corpus[doc].items():
#             lengths.append(len(section))

#     Lave[target] = sum(lengths) / len(lengths)

In [13]:







def tat_score(query, docIds, docs):
    # Implementation of how to calculate tat score function
    query_vector = np.ones(len(query))
    query_vector = query_vector/np.linalg.norm(query_vector)
    #fds

    documents_vectors = defaultdict(lambda: [])

    for term in query:
        all_document_ids = set(doc_ids)
        for doc in docs[term]['title']:
            documents_vectors[doc].append((1 + np.log(len(doc))) * get_idf(term))
            # * get_idf(term, docs))
            all_document_ids.discard(doc)
        for doc in all_document_ids:
            documents_vectors[doc].append(0)

    for doc in documents_vectors:
        v = np.asarray(documents_vectors[doc])
        if v.any():
            v /= np.linalg.norm(v)

        documents_vectors[doc] = v

    scores = {doc: np.dot(v, query_vector) for doc, v in documents_vectors.items()}

    return scores


def dat_score(target, query, docIds, docs):
    # Implementation of how to calculate dat score function
    query_vector = np.ones(len(query))
    query_vecor /= np.linalg.norm(query_vecor)

    documents_vectors = defaultdict(lambda: [])

    for doc in doc_ids:
        vector = [(1 + np.log(len(docs[term][target][doc]))) * get_idf(term,docs) for term in query]
        documents_vectors[doc] = np.asarray(vector)

    for doc in documents_vectors:
        v = np.asarray(documents_vectors[doc])
        if v.any():
            v /= np.linalg.norm(v)

        documents_vectors[doc] = v

    scores = {doc: np.dot(v, query_vecor) for doc, v in documents_vectors.items()}

    return scores

def okapi_score(target, query, docIds, docs):
    targets = ['title', 'abstract']
    Lave = {}

    for target in targets:
      lengths = []

    for doc in docIds:
        for _, section in corpus[doc].items():
            lengths.append(len(section))

    Lave[target] = sum(lengths) / len(lengths)

    # Implementation of how to calculate okapi score
    scores = {}
    k1 = 1.2
    b = 0.75

    for doc in doc_ids:
        score = 0
        if target in corpus[doc].keys():
            for term in query:
                tf = len(docs[term][target][doc])
                Ld = len(corpus[doc][target])
                # print(Lave)
                score += get_idf(term) * ((k1 + 1) * tf) / (k1 * (1 - b + b * (Ld / Lave[target])) + tf)

        scores[doc] = score

    return scores

def search(query, max_result_count, method='ltc-lnc-tat', weight=0.5):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    method: 'ltc-lnc-tat' or 'ltc-lnc-dat' or 'okapi25'

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retrieved documents with snippet
    """

    p = Preprocessor.get_instance(None)
    query = p.preprocess(query)

    methods = {
        'ltc-lnc-tat': tat_score,
        'ltc-lnc-dat': dat_score,
        'okapi25': okapi_score
    }

    if method not in methods:
        raise ValueError("Undefined method")

    calculate_function = methods[method]
    target_values = ['title', 'abstract']
    scores = {target: defaultdict(lambda: 0) for target in target_values}

    #fds
    for target in target_values:
        if method == "okapi25" or method == "ltc-lnc-dat":
          scores[target] = calculate_function(target, query, doc_ids, docs)
        # elif method == "ltc-lnc-dat":
        #   scores[target] = calculate_function(target, query)
        else:
          scores[target] = calculate_function(query, doc_ids, docs)

    sorted_scores = [x for nn, x in sorted([
        (weight * scores[target_values[0]][doc] + (1 - weight) * scores[target_values[1]][doc], doc)
        for doc in doc_ids], reverse=True)]

    if max_result_count == -1:
        return sorted_scores

    return sorted_scores[:max_result_count]

<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه
smoothing
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد.
</p>
</div>

In [14]:
def calculate_cf_T(targets, docs):
  T = 0
  cf = defaultdict(lambda: 0)
# targets = ['abstract', 'title']

  for token, token_data in docs.items():
    for target in targets:
        for doc in token_data[target]:
            cf[token] += len(token_data[target][doc])

  T = sum(cf.values())

  return cf, T
# Calculate cf and T
target_values = ['title', 'abstract']
cf, T = calculate_cf_T(target_values, docs)

In [15]:
def lm_search(query: str, max_result_count: int, smoothing_method='JM'):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retrieved documents with snippet
    """

    if smoothing_method not in ['JM', 'D']:
        raise ValueError("Undefined smoothing method")

    p = Preprocessor(None)
    query = p.preprocess(query)
    lamda = 0.5
    alpha = 0.1

    probs = defaultdict(lambda: 1.0)
    priors = defaultdict(lambda: 1.0)

    for term in query:
        priors[term] = cf[term] / T

    for doc in doc_ids:
        Ld = sum(len(section) for _, section in corpus[doc].items())
        for term in query:
            tf = get_tf(term, docs, doc)
            if smoothing_method == 'JM':
                probs[doc] *= lamda * priors[term] + (1 - lamda) * tf / Ld
            else:
                probs[doc] *= (tf + alpha * priors[term]) / (Ld + alpha)

    sorted_scores = [x for _, x in sorted([(probs[doc], doc) for doc in doc_ids], reverse=True)]

    if max_result_count == -1:
        return sorted_scores

    return sorted_scores[:max_result_count]

<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید.
</p>
</div>

In [16]:
def mean(a):
    return sum(a) / len(a)

class Evaluation:
    def __init__(self, actual, predict):
        self.actual = actual
        self.predict = predict

    def evaluate(self, require_scores):
        """
        Prints require scores for actual and predicts array

        Parameters
        ----------
        require_scores : List[str]
            Scores required to be calculated

        Returns
        -------
        """
        scores_functions = {'precision': self.precision,
                            'recall': self.recall,
                            'f1': self.f1,
                            'map': self.map_,
                            'ndcg': self.ndcg,
                            'mrr': self.mrr}

        for score in require_scores:
            print(f"{score}: {scores_functions[score]()}")

    def precision(self):
        precision = []
        for i in range(len(self.actual)):
            precision.append(sum(1 for p in list(self.predict[i]) if p in self.actual[i]) / len(self.predict[i]))
        return mean(precision)

    def recall(self):
        recall = []
        for i in range(len(self.actual)):
            recall.append(sum(1 for a in self.actual[i] if a in self.predict[i]) / len(self.actual[i]))
        return mean(recall)

    def f1(self):
        pr = self.precision()
        re = self.recall()
        f1_score = 2 * (pr * re) / (pr + re)
        return f1_score

    def map_(self):
        map_scores = []
        for i in range(len(self.actual)):
            predicted = self.predict[i]
            cnt = 0
            precisions = []
            visited = False
            for j, doc in enumerate(predicted):
                if doc in self.actual[i]:
                    cnt += 1
                    precisions.append(cnt / (j + 1))
                    visited = True
            if visited:
                map_scores.append(mean(precisions))
            else:
                map_scores.append(0)
        return mean(map_scores)

    def ndcg(self):
        def dcg(rel):
            return np.sum((2**r - 1) / np.log2(idx + 2) for idx, r in enumerate(rel))

        ndcg_scores = []
        for i in range(len(self.actual)):
            predicted = self.predict[i]
            rel = [1 if p in self.actual[i] else 0 for p in predicted]
            ideal_rel = [1] * len(self.actual[i])
            ndcg_scores.append(dcg(rel) / dcg(ideal_rel))
        return mean(ndcg_scores)

    def mrr(self):
        mrr_scores = []
        for i in range(len(self.actual)):
            for j, doc in enumerate(self.predict[i]):
                if doc in self.actual[i]:
                    mrr_scores.append(1.0 / (j + 1))
                    break
            else:
                mrr_scores.append(0)
        return mean(mrr_scores)



In [17]:

import json
require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

# Read actuals and queries from validation.json file
actuals = []
queries = []

with open("validation.json", 'r') as f:
    test_data = json.load(f)

queries = list(test_data.keys())
actuals = [test_data[query] for query in queries]

In [18]:
predicts = [search(query, 10, method='ltc-lnc-tat') for query in queries]

eval_vsm = Evaluation(actuals, predicts)
eval_vsm.evaluate(require_scores)

precision: 0.55
recall: 0.55
f1: 0.55
map: 0.7147321428571428
ndcg: 0.5784242620410707
mrr: 0.75


<ipython-input-16-d6f4447864d2>:69: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum((2**r - 1) / np.log2(idx + 2) for idx, r in enumerate(rel))


In [20]:
predicts = [search(query, 10, method='okapi25') for query in queries]

eval_bm = Evaluation(actuals, predicts)
eval_bm.evaluate(require_scores)

precision: 0.5499999999999999
recall: 0.5499999999999999
f1: 0.5499999999999999
map: 0.6875661375661375
ndcg: 0.5796797760974072
mrr: 0.7142857142857143


<ipython-input-16-d6f4447864d2>:69: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum((2**r - 1) / np.log2(idx + 2) for idx, r in enumerate(rel))


In [21]:
predicts = [lm_search(query, 10) for query in queries]

eval_lm = Evaluation(actuals, predicts)
eval_lm.evaluate(require_scores)

precision: 0.6333333333333334
recall: 0.6333333333333334
f1: 0.6333333333333334
map: 0.732796884185773
ndcg: 0.6348826605782925
mrr: 0.7833333333333333


<ipython-input-16-d6f4447864d2>:69: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum((2**r - 1) / np.log2(idx + 2) for idx, r in enumerate(rel))
